<a href="https://colab.research.google.com/github/wassimchouchen/Automatic-Speech-Recognition-/blob/main/Language_style_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
import pydub

from sklearn.cluster import AgglomerativeClustering
import numpy as np


In [2]:
num_speakers = 2 
language = 'any' 
model_size = 'large'

In [3]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving conv.wav to conv.wav


In [4]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
# Load the audio file
audio_file = pydub.AudioSegment.from_wav(path)
audio_file = audio_file.set_channels(1)
audio_file.export(path, format="wav")
with wave.open(path) as f:
  print(f.getnchannels())

1


In [5]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.87G/2.87G [00:36<00:00, 83.9MiB/s]


In [6]:
result = model.transcribe(path)
segments = result["segments"]

In [7]:
text=result["text"]

In [8]:
text

" podcastfrancaisfacile.com Bonjour. Bonjour. Installez-vous je vous en prie. Merci. Alors qu'est-ce qui vous amène ? Eh bien j'ai des douleurs au ventre, au niveau de l'estomac. Vous avez mal à l'estomac ? En fait je ne suis pas sûr que ce soit à l'estomac. Vous avez mal en permanence ? Non, ce n'est pas tout le temps, c'est surtout après manger. Comment est la douleur ? Ce n'était pas très fort au début, mais depuis quelques jours c'est très aigu. Quand ont commencé les premières douleurs ? C'était il y a deux mois environ. Je pensais que ça passerait, mais finalement c'est de pire en pire. Bien, on va regarder ça. Enlevez votre chemise et allongez-vous."

In [ ]:
!pip install -U transformers

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [11]:
print(summarizer(text))

[{'summary_text': "J'ai des douleurs au ventre, au niveau of l'estomac. C'était il y a deux mois environ. Je pensais que ça passerait, mais finalement c'est de pire en pire. Enlevez votre chemise et allongez-vous."}]


In [ ]:
text=" podcastfrancaisfacile.com Bonjour. Bonjour. Installez-vous je vous en prie. Merci. Alors qu'est-ce qui vous amène ? Eh bien j'ai des douleurs au ventre, au niveau de l'estomac. Vous avez mal à l'estomac ? En fait je ne suis pas sûr que ce soit à l'estomac. Vous avez mal en permanence ? Non, ce n'est pas tout le temps, c'est surtout après manger. Comment est la douleur ? Ce n'était pas très fort au début, mais depuis quelques jours c'est très aigu. Quand ont commencé les premières douleurs ? C'était il y a deux mois environ. Je pensais que ça passerait, mais finalement c'est de pire en pire. Bien, on va regarder ça. Enlevez votre chemise et allongez-vous."

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")
model = AutoModelForSeq2SeqLM.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")

In [13]:
encoded_text = tokenizer(text, return_tensors="pt")
summary_ids = model.generate(**encoded_text)
sum=tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


In [14]:
sum

"J'ai des douleurs au ventre au niveau de l'estomac."

# Agressivité

In [ ]:
from transformers import AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-imdb")
model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-imdb")
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
print(predicted_class_id)
if predicted_class_id == 1 :
  print("discour pas aggresive:",logits[0])
else :
  print("discour  aggresive:",logits[0])


In [50]:
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
print(predicted_class_id)
if predicted_class_id == 1 :
  print("discour pas aggresive:",logits[0])
else :
  print("discour  aggresive:",logits[0])


1
discour pas aggresive: tensor([-0.9735,  0.6360])


In [51]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

In [60]:
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

for i,j in enumerate(np.array(logits[0])):
  print(model.config.id2label[i],j)

joy -0.11674911
optimism 0.7449858
anger 0.7546211
sadness -0.44103074


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-offensive")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-offensive")

In [63]:
inputs = tokenizer("C'est un député très agressif historiquement", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
for i,j in enumerate(np.array(logits[0])):
  print(model.config.id2label[i],j)

non-offensive 0.7593189
offensive -0.78242993
